In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
base.reset_seed()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "sst2"

In [ ]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

In [8]:
vocab = base.get_vocab(all_data_tokens)

In [9]:
word_index = dict(zip(vocab, range(len(vocab))))

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [ ]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

14621


In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 14305 words (316) misses


In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [ ]:
num_epochs = 15
batch_size = 128

In [17]:
#Nápočet epoch na steps
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [20]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=True)

In [ ]:
base.reset_seed()

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base_hp-search", epochs=num_epochs, batch_size=batch_size)

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

In [24]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base",
    n_trials=150
)

[I 2025-03-23 01:15:54,583] A new study created in memory with name: Base


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.422600,0.434365,0.788991,0.797190,0.790477,0.788053
2,0.337800,0.419709,0.809633,0.812991,0.808643,0.808727
3,0.302600,0.417057,0.809633,0.811507,0.810327,0.809533
4,0.273800,0.473746,0.811927,0.823243,0.810179,0.809619
5,0.242300,0.436711,0.827982,0.828483,0.828345,0.827978
6,0.215700,0.468983,0.829128,0.833631,0.828039,0.828174
7,0.193900,0.435610,0.832569,0.833693,0.832007,0.832216
8,0.176500,0.484555,0.846330,0.847529,0.845773,0.846006
9,0.160000,0.524556,0.839450,0.839415,0.839522,0.839428
10,0.147100,0.504254,0.829128,0.835197,0.830355,0.828672


[I 2025-03-23 01:17:35,258] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 6}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.397700,0.447983,0.783257,0.798160,0.785268,0.781275
2,0.308400,0.401327,0.827982,0.831937,0.826955,0.827108
3,0.249000,0.422298,0.830275,0.834983,0.829166,0.829297
4,0.200900,0.454170,0.842890,0.843640,0.842437,0.842636
5,0.160400,0.449861,0.847477,0.848364,0.847952,0.847461


[I 2025-03-23 07:19:47,791] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0005279934990252418, 'weight_decay': 0.009000000000000001, 'warmup_steps': 34}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.404400,0.439796,0.784404,0.793934,0.786015,0.783236
2,0.315100,0.412896,0.827982,0.830983,0.827082,0.827269
3,0.262500,0.421345,0.824541,0.827955,0.823577,0.823734
4,0.219600,0.444765,0.833716,0.835139,0.833091,0.833310
5,0.181500,0.436488,0.848624,0.848569,0.848657,0.848595
6,0.149400,0.512193,0.847477,0.852694,0.846352,0.846571
7,0.124800,0.479655,0.845183,0.847660,0.844395,0.844652
8,0.104100,0.506412,0.864679,0.865350,0.865096,0.864672
9,0.084600,0.642493,0.837156,0.840867,0.838111,0.836936
10,0.070500,0.669354,0.839450,0.842854,0.840364,0.839259


[I 2025-03-23 07:21:38,067] Trial 149 pruned. 


In [25]:
print(best_trial)

BestRun(run_id='88', objective=0.8749406267968206, hyperparameters={'learning_rate': 0.0007832919195971522, 'weight_decay': 0.008, 'warmup_steps': 31}, run_summary=None)


In [ ]:
base.reset_seed()

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [31]:
best_trial2 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-23 07:21:38,433] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.919000,1.643253,0.784404,0.792961,0.785931,0.783378
2,1.426500,1.467759,0.816514,0.823376,0.815147,0.815034
3,1.239100,1.380690,0.818807,0.824863,0.817525,0.817493
4,1.091900,1.419749,0.814220,0.821022,0.812853,0.812722
5,0.953500,1.315268,0.836009,0.836032,0.836143,0.835999


[I 2025-03-23 07:22:45,368] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 38, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.452400,1.282989,0.826835,0.830298,0.827766,0.826616
2,0.784400,1.278039,0.841743,0.844052,0.840974,0.841221
3,0.445900,1.228873,0.850917,0.851349,0.850573,0.850741
4,0.265200,1.370055,0.842890,0.848427,0.841721,0.841900
5,0.173000,1.403558,0.840596,0.843909,0.839680,0.839912


[I 2025-03-23 15:42:54,745] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0006627853536758961, 'weight_decay': 0.0, 'warmup_steps': 0, 'lambda_param': 0.9, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.728100,1.679653,0.795872,0.805724,0.797487,0.794766
2,1.229600,1.278475,0.830275,0.830239,0.830176,0.830203
3,0.944100,1.382887,0.833716,0.840343,0.832418,0.832476
4,0.725800,1.369946,0.837156,0.840583,0.836217,0.836432
5,0.572200,1.327224,0.856651,0.857110,0.857003,0.856650
6,0.455700,1.389737,0.853211,0.853162,0.853162,0.853162
7,0.374300,1.450515,0.842890,0.844167,0.842311,0.842542
8,0.302900,1.389971,0.864679,0.865870,0.865223,0.864653
9,0.253100,1.570562,0.848624,0.850917,0.849373,0.848527
10,0.212600,1.394859,0.863532,0.864820,0.864097,0.863502


[I 2025-03-23 15:48:07,837] Trial 149 finished with value: 0.8703062418477439 and parameters: {'learning_rate': 0.0006627853536758961, 'weight_decay': 0.0, 'warmup_steps': 0, 'lambda_param': 0.9, 'temperature': 4.0}. Best is trial 100 with value: 0.881824628991491.


In [ ]:
print(best_trial2)

BestRun(run_id='100', objective=0.881824628991491, hyperparameters={'learning_rate': 0.0008284065855808625, 'weight_decay': 0.002, 'warmup_steps': 42, 'lambda_param': 0.9, 'temperature': 5.5}, run_summary=None)


In [ ]:
#Nápočet epoch na steps
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [ ]:
base.reset_seed()

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base_aug_hp-search", epochs=num_epochs, batch_size=batch_size)

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [ ]:
best_trial3 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug",
    n_trials=150
)

[I 2025-03-23 15:48:08,187] A new study created in memory with name: Base-aug


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 305}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.301100,0.412027,0.833716,0.838676,0.832586,0.832728
2,0.168400,0.438308,0.847477,0.847738,0.847194,0.847331
3,0.107400,0.616542,0.831422,0.833508,0.830671,0.830888
4,0.073300,0.642944,0.839450,0.839395,0.839395,0.839395
5,0.053000,0.727546,0.852064,0.852087,0.852204,0.852055


[I 2025-03-23 15:59:40,554] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 65}


...

Trial 44 with params: {'learning_rate': 0.0034104651800238926, 'weight_decay': 0.01, 'warmup_steps': 297}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.184600,0.486978,0.860092,0.860206,0.860297,0.860089
2,0.070500,0.638008,0.850917,0.850931,0.850783,0.850839
3,0.048100,0.720432,0.853211,0.853325,0.853414,0.853208
4,0.037100,0.723554,0.857798,0.858131,0.857498,0.857651
5,0.029300,0.884021,0.854358,0.854698,0.854667,0.854358
6,0.024000,0.964237,0.857798,0.857798,0.857919,0.857786
7,0.019200,1.030546,0.856651,0.857161,0.856287,0.856470
8,0.016200,1.154444,0.842890,0.843054,0.842648,0.842761
9,0.012700,1.226750,0.856651,0.856891,0.856919,0.856651
10,0.009800,1.583062,0.853211,0.853157,0.853246,0.853183


[I 2025-03-24 07:30:06,344] Trial 44 finished with value: 0.8496660547078061 and parameters: {'learning_rate': 0.0034104651800238926, 'weight_decay': 0.01, 'warmup_steps': 297}. Best is trial 25 with value: 0.8635319306191749.


Trial 45 with params: {'learning_rate': 0.001112407136080856, 'weight_decay': 0.009000000000000001, 'warmup_steps': 287}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.219400,0.462439,0.858945,0.859227,0.858666,0.858810


In [43]:
print(best_trial3)

BestRun(run_id='88', objective=0.783874708715165, hyperparameters={'learning_rate': 0.0034351095517178363, 'weight_decay': 0.009000000000000001, 'warmup_steps': 15}, run_summary=None)


In [31]:
base.reset_seed()

In [32]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill_aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill_aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

In [33]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

In [34]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [35]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

In [36]:
best_trial4 = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug",
    n_trials=150
)

In [37]:
print(best_trial4)

BestRun(run_id='69', objective=0.8232465493473291, hyperparameters={'learning_rate': 0.003663666314607629, 'weight_decay': 0.0, 'warmup_steps': 21, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}, run_summary=None)


In [38]:
print("Best normal training score: ", best_trial)
print("Best distilation trianing score: ", best_trial2)
print("Best normal training score with augmentations: ", best_trial3)
print("Best distilation trianing score with augmentations: ",best_trial4)

NameError: name 'best_trial' is not defined